In [1]:
import numpy as np 
import pandas as pd 
import math 
from sklearn import datasets, model_selection, naive_bayes
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
%matplotlib inline

Load datasets:

In [2]:
WildFiresData = pd.read_csv('Historical_Wildfires.csv')
HistoricalWeather = pd.read_csv('HistoricalWeather.csv')

Create train data and labels(it takes 7-10 minutes):

In [3]:
data_train = []
data_train_SoilPer = []
data_train_SolarPer = []
data_train_SolarSoil = []
data_train_Per = []
data_train_Soil = []
data_train_Solar = []
data_train_Empty = []
label_train = []
number_emptydata = 0
for i in range(len(WildFiresData)):
    dateoffire = WildFiresData['Date'].iloc[i]
    region = WildFiresData['Region'].iloc[i]
    spliteddate = dateoffire.split('/')
    if int(spliteddate[0]) < 10:
        spliteddate[0] = '0' + spliteddate[0]
    if int(spliteddate[1]) < 10:
        spliteddate[1] = '0' + spliteddate[1]
    finaldate = spliteddate[2] + '-' + spliteddate[1] + '-' + spliteddate[0]
    matchdate = HistoricalWeather[HistoricalWeather['Date'] == finaldate]
    final_matcheddate = matchdate[matchdate['Region'] == region]
    try:
        Precipitation = final_matcheddate[final_matcheddate['Parameter'] == 'Precipitation']['mean()'].iloc[0]
        RelativeHumidity = final_matcheddate[final_matcheddate['Parameter'] == 'RelativeHumidity']['mean()'].iloc[0]
        SoilWaterContent = final_matcheddate[final_matcheddate['Parameter'] == 'SoilWaterContent']['mean()'].iloc[0]
        SolarRadiation = final_matcheddate[final_matcheddate['Parameter'] == 'SolarRadiation']['mean()'].iloc[0]
        Temperature = final_matcheddate[final_matcheddate['Parameter'] == 'Temperature']['mean()'].iloc[0]
        WindSpeed = final_matcheddate[final_matcheddate['Parameter'] == 'WindSpeed']['mean()'].iloc[0]
        data_train.append([Precipitation, RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed])
        data_train_SoilPer.append([RelativeHumidity, SolarRadiation, Temperature, WindSpeed])
        data_train_SolarPer.append([RelativeHumidity, SoilWaterContent, Temperature, WindSpeed])
        data_train_SolarSoil.append([Precipitation, RelativeHumidity, Temperature, WindSpeed])
        data_train_Per.append([RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed])
        data_train_Soil.append([Precipitation, RelativeHumidity, SolarRadiation, Temperature, WindSpeed])
        data_train_Solar.append([Precipitation, RelativeHumidity, SoilWaterContent, Temperature, WindSpeed])
        data_train_Empty.append([RelativeHumidity, Temperature, WindSpeed])
        label_train.append(1)
    except:
        number_emptydata += 1 
    try:
        allregion = matchdate['Region'].drop_duplicates().values
        allregion = np.delete(allregion, np.where(allregion == region))
        for tempregion in allregion:
            final_matchedlabeldate = matchdate[matchdate['Region'] == tempregion]
            Precipitation = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'Precipitation']['mean()'].iloc[0]
            RelativeHumidity = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'RelativeHumidity']['mean()'].iloc[0]
            SoilWaterContent = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'SoilWaterContent']['mean()'].iloc[0]
            SolarRadiation = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'SolarRadiation']['mean()'].iloc[0]
            Temperature = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'Temperature']['mean()'].iloc[0]
            WindSpeed = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'WindSpeed']['mean()'].iloc[0]
            data_train.append([Precipitation, RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed])
            data_train_SoilPer.append([RelativeHumidity, SolarRadiation, Temperature, WindSpeed])
            data_train_SolarPer.append([RelativeHumidity, SoilWaterContent, Temperature, WindSpeed])
            data_train_SolarSoil.append([Precipitation, RelativeHumidity, Temperature, WindSpeed])
            data_train_Per.append([RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed])
            data_train_Soil.append([Precipitation, RelativeHumidity, SolarRadiation, Temperature, WindSpeed])
            data_train_Solar.append([Precipitation, RelativeHumidity, SoilWaterContent, Temperature, WindSpeed])
            data_train_Empty.append([RelativeHumidity, Temperature, WindSpeed])
            label_train.append(0)
    except:  
        number_emptydata += 1 

Use Machine Learning with trained data and labels to predict wildfire:

In [45]:
def choosingmachine(Existence_list, Precipitation, RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed):    
    model = MultinomialNB()
    if Existence_list == ['Yes','No','No']:
        X = data_train_SoilPer
        y = label_train
        model.fit(X, y)
        predictML = round(model.predict_proba([[RelativeHumidity, SolarRadiation, Temperature, WindSpeed]])[0][1]*100,2)
        accuracyML = round(model.score(X, y)*100,2)
            
    elif Existence_list == ['No','Yes','No']:
        X = data_train_SolarPer
        y = label_train
        model.fit(X, y)
        predictML = round(model.predict_proba([[RelativeHumidity, SoilWaterContent, Temperature, WindSpeed]])[0][1]*100,2)
        accuracyML = round(model.score(X, y)*100,2)
        
    elif Existence_list == ['No','No','Yes']:
        X = data_train_SolarSoil
        y = label_train
        model.fit(X, y)
        predictML = round(model.predict_proba([[Precipitation, RelativeHumidity, Temperature, WindSpeed]])[0][1]*100,2)
        accuracyML = round(model.score(X, y)*100,2)
        
    elif Existence_list == ['Yes','Yes','No']:
        X = data_train_Per
        y = label_train
        model.fit(X, y)
        predictML = round(model.predict_proba([[RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed]])[0][1]*100,2)
        accuracyML = round(model.score(X, y)*100,2)
        
    elif Existence_list == ['Yes','No','Yes']:
        X = data_train_Soil
        y = label_train
        model.fit(X, y)
        predictML = round(model.predict_proba([[Precipitation, RelativeHumidity, SolarRadiation, Temperature, WindSpeed]])[0][1]*100,2)
        accuracyML = round(model.score(X, y)*100,2)
        
    elif Existence_list == ['No','Yes','Yes']:
        X = data_train_Solar
        y = label_train
        model.fit(X, y)
        predictML = round(model.predict_proba([[Precipitation, RelativeHumidity, SoilWaterContent, Temperature, WindSpeed]])[0][1]*100,2)
        accuracyML = round(model.score(X, y)*100,2)
    
    elif Existence_list == ['No', 'No', 'No']:
        X = data_train_Empty
        y = label_train
        model.fit(X, y)
        predictML = round(model.predict_proba([[RelativeHumidity, Temperature, WindSpeed]])[0][1]*100,2)
        accuracyML = round(model.score(X, y)*100,2)    
        
    else:
        X = data_train
        y = label_train
        model.fit(X, y)
        predictML = round(model.predict_proba([[Precipitation, RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed]])[0][1]*100,2)
        accuracyML = round(model.score(X, y)*100,2)
    return [accuracyML, predictML] 

FFWI formula to predict fire index: 

In [46]:
def FFWIformula(windavr, tempavr, havr):
    tempavr = (tempavr * (9/5)) + 32
    windavr = windavr/0.44704
    if havr < 10:
        m = 0.03229 + (0.281073 * havr)-(0.000578 * havr * tempavr)
    elif 10 <= havr <= 50:
        m = 2.22749 + (0.160107 * havr) - (0.01478 * tempavr)
    elif havr > 50:
        m = 21.0606 + (0.005565 * havr**2) - (0.00035 * havr * tempavr) - (0.483199 * havr)
    n = 1 - (2 * (m/30)) + (1.5 * (m/30)**2) - (0.5 * (m/30)**3)
    FFWI = n * (math.sqrt(1 + windavr**2)/0.3002)
    return round(FFWI, 2)

The function to find average probability:

In [47]:
def AverageProb():
    averageProb = (choosingmachine(Existence_list, Precipitation, RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed)[1] + FFWIformula(WindSpeed, Temperature, RelativeHumidity))/2
    if averageProb < 50:
        dangerlavel = 'Low'
    elif 50 <= averageProb < 70:
        dangerlavel = 'Medium'
    elif 70 <= averageProb < 90:
        dangerlavel = 'High (dangerous!)'
    elif 90 <= averageProb <= 100:
        dangerlavel = 'Extreme (Very dangerous!)'
    elif 101 <= averageProb:
        dangerlavel = 'Very Extreme (Extremely dangerous!)'
    return dangerlavel

The function to ask weather parameters based on choice of an user:

In [48]:
def Inputchoice(Existence_list):
    if Existence_list == ['Yes','No','No']:
        Precipitation = 0
        SoilWaterContent = 0
        SolarRadiation = float(input('mean solar radiation(MJ/day):'))

    
    elif Existence_list == ['No','Yes','No']:
        Precipitation = 0
        SoilWaterContent = float(input('mean soil water content(m**3/m**-3):'))
        SolarRadiation = 0
        
    elif Existence_list == ['No','No','Yes']:
        Precipitation = float(input('mean precipitation(in):'))
        SoilWaterContent = 0
        SolarRadiation = 0

        
    elif Existence_list == ['Yes','Yes','No']:
        Precipitation = 0
        SoilWaterContent = float(input('mean soil water content(m**3/m**-3):'))
        SolarRadiation = float(input('mean solar radiation(MJ/day):'))

        
    elif Existence_list == ['Yes','No','Yes']:
        Precipitation = float(input('mean precipitation(in):'))
        SoilWaterContent = 0
        SolarRadiation = float(input('mean solar radiation(MJ/day):'))

        
    elif Existence_list == ['No','Yes','Yes']:
        Precipitation = float(input('mean precipitation(in):'))
        SoilWaterContent = float(input('mean soil water content(m**3/m**-3):'))
        SolarRadiation = 0
    
    elif Existence_list == ['No', 'No', 'No']:
        Precipitation = 0
        SoilWaterContent = 0
        SolarRadiation = 0
        
    else:
        Precipitation = float(input('mean precipitation(in):'))
        SoilWaterContent = float(input('mean soil water content(m**3/m**-3):'))
        SolarRadiation = float(input('mean solar radiation(MJ/day):'))
    return [Precipitation, SoilWaterContent, SolarRadiation]

The function to predict wildfires probability based on given DataFrame:

In [49]:
def predictDataFrame(data_csv):
    data_accuracy = []
    data_predict = []
    for i in range(len(data_csv)):
        Existence_Solar = 'Yes'
        Existence_Soil = 'Yes'
        Existence_Per = 'Yes'
        try:
            Precipitation = data_csv['Mean Precipitation'].iloc[i]
        except:
            Existence_Per = 'No'
            Precipitation = 0
        try:
            SoilWaterContent = data_csv['Mean Soil Water Content'].iloc[i]
        except:
            Existence_Soil = 'No'
            SoilWaterContent = 0
        try:
            SolarRadiation = data_csv['Mean Solar Radiation'].iloc[i]
        except:
            Existence_Solar = 'No'     
            SolarRadiation = 0
        RelativeHumidity = data_csv['Mean Relative Humidity'].iloc[i]
        Temperature = data_csv['Mean Temperature'].iloc[i]
        WindSpeed = data_csv['Mean Wind Speeds'].iloc[i]
        Existence_list = [Existence_Solar, Existence_Soil, Existence_Per] 
        accuracyML, predictML = choosingmachine(Existence_list, Precipitation, RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed)
        data_predict.append(predictML)
        data_accuracy.append(accuracyML)
    return [data_predict, data_accuracy]

The function to save results from the predicted DF and plot:

In [66]:
def saveResults():
    if input('Do you want to save result in csv? (Yes/No) ') == 'Yes':
        createDFresult().to_csv('prediction_result.csv')
        print('The data saved as prediction_result.csv!')
    if input('Do you want to save plot in png? (Yes/No)? ') == 'Yes':
        createDFresult().plot()
        plt.savefig('prediction_plot.png', dpi = 300, bbox_inches = 'tight')
        print('The plot saved as prediction_plot.png! ')

Print results:

In [65]:
def printresults():
    print('Probability of occuring fire by Machine Learing(%): ', predictML)
    print('Fire index by FFWI formula: ', FFWIformula(WindSpeed, Temperature, RelativeHumidity))
    print('Danger level by average probability of FFWI formula and ML: ', AverageProb())
    print('Accuracy of ML: ', accuracyML)

Create DataFrame based on ML results:

In [63]:
def createDFresult():
    res_data = data_csv.drop(columns = ['Mean Precipitation', 'Mean Relative Humidity', 'Mean Soil Water Content', 'Mean Solar Radiation', 'Mean Temperature', 'Mean Wind Speeds'])
    data_predict, data_accuracy = predictDataFrame(data_csv)
    res_data['Prediction'] = data_predict
    res_data['Accuracy'] = data_accuracy
    res_data = res_data.set_index('Date (mm/dd/yy)')
    return res_data 

Create plot based on ML results:

In [64]:
def createDFplot():
    createDFresult().plot(kind = 'line', y = ['Prediction','Accuracy'], figsize=(20, 10))
    plt.ylabel('Prediction')
    plt.show()    

Run Prediction Code:

In [68]:
if input('Do you want to use DataFrame? (Yes/No) ') == 'Yes':
    data_csv = pd.read_csv(input('Put csv name: '))
    print(createDFresult())
    createDFplot()
    saveResults()

else:        
    print('Which types of weather parametrs do you have? (Less parameters will affect the accuracy of Machine Learning)')
    Existence_Solar = input('Solar Radiation (Yes/No)')
    Existence_Soil = input('Soil Water Content (Yes/No)')
    Existence_Per = input('Precipitation (Yes/No)')
    Existence_list = [Existence_Solar, Existence_Soil, Existence_Per]

    WindSpeed = float(input('mean wind (m/s):'))
    Temperature = float(input('mean temperature (C):'))
    RelativeHumidity = float(input('mean humidity (%):'))
    
    Precipitation, SoilWaterContent, SolarRadiation = Inputchoice(Existence_list)
    accuracyML, predictML = choosingmachine(Existence_list, Precipitation, RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed)
    
    printresults()

Do you want to use DataFrame? (Yes/No) No
Which types of weather parametrs do you have? (Less parameters will affect the accuracy of Machine Learning)
Solar Radiation (Yes/No)Yes
Soil Water Content (Yes/No)No
Precipitation (Yes/No)Yes
mean wind (m/s):5
mean temperature (C):20
mean humidity (%):20
mean precipitation(in):0.04
mean solar radiation(MJ/day):25
Probability of occuring fire by Machine Learing(%):  72.6
Fire index by FFWI formula:  27.53
Danger level by average probability of FFWI formula and ML:  Medium
Accuracy of ML:  71.31


In [69]:
res_data

,Prediction,Accuracy
Date (mm/dd/yy),,
10/22/21,47.76,71.23
10/23/21,73.86,71.23
10/24/21,16.35,71.23
10/25/21,82.54,71.23
10/26/21,0.00,71.23
10/27/21,1.44,71.23
10/28/21,94.63,71.23
